In [1]:
import json
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE 
from sklearn.utils import resample
import pandas as pd

from epilepsypcm.utils.make_df import make_df, get_df_list, concat_dfs, class_balance
from epilepsypcm.utils.outcome_params import engel_score

In [2]:
newdf = pd.read_csv("newDF.csv").iloc[: , 1:]
print(newdf)

          Channels  outcome   SigResp   SigStim  N1RespAvg  N1RespSDV  \
0         LA9_LA10      0.0  0.000000  0.250000   1.530459   1.134169   
1        LAH1_LAH2      1.0  0.076923  0.250000   7.062089  20.678040   
2        LAH8_LAH9      0.0  0.307692  0.125000   4.032683   2.995624   
3        LPH1_LPH2      0.0  0.115385  0.333333   6.522655  17.219698   
4        LPH7_LPH8      0.0  0.307692  0.083333   7.304854  10.728727   
..             ...      ...       ...       ...        ...        ...   
304    RMBT2_RMBT3      0.0  0.250000  0.375000  10.029460  19.122649   
305  RPTI19_RPTI20      0.0  0.250000  0.250000  13.323164  20.134989   
306  RPTS60_RPTS61      0.0  0.125000  0.250000   9.883556  20.481622   
307    RPFS4_RPFS5      0.0  0.000000  0.000000   2.462636   1.320800   
308  RPTI26_RPTI27      0.0  0.250000  0.250000  14.681348  23.509883   

     N2RespAvg  N2RespSDV  P2RespAvg  P2RespSDV  N1StimAvg  N1StimSDV  \
0     2.040546   1.369091   1.304096   0.892887   

In [3]:
y = newdf['outcome']
X = newdf.drop(columns = 'outcome')

# Random Oversampler

In [4]:
from collections import Counter
print(Counter(y))
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X, y)
# summarize class distribution
print(Counter(y_over))

Counter({0.0: 268, 1.0: 41})
Counter({0.0: 268, 1.0: 268})


In [5]:
X_over

,Channels,SigResp,SigStim,N1RespAvg,N1RespSDV,N2RespAvg,N2RespSDV,P2RespAvg,P2RespSDV,N1StimAvg,N1StimSDV,N2StimAvg,N2StimSDV,P2StimAvg,P2StimSDV
0,LA9_LA10,0.000000,0.250000,1.530459,1.134169,2.040546,1.369091,1.304096,0.892887,7.178516,10.790559,6.536859,6.383906,5.785279,7.331337
1,LAH1_LAH2,0.076923,0.250000,7.062089,20.678040,3.320937,2.795194,2.178348,2.563587,7.883750,16.334814,4.921208,3.029861,2.352152,1.803711
2,LAH8_LAH9,0.307692,0.125000,4.032683,2.995624,3.450575,2.883998,2.376435,2.139730,2.505806,2.164878,2.190308,1.489418,2.033410,2.112725
3,LPH1_LPH2,0.115385,0.333333,6.522655,17.219698,3.369771,3.506524,2.281413,3.303814,6.505212,8.264811,4.630948,3.574869,4.187831,3.626223
4,LPH7_LPH8,0.307692,0.083333,7.304854,10.728727,4.610356,4.159990,3.468193,2.645476,2.823084,2.925869,1.606341,1.075715,1.461462,1.012732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,LIPF3_LIPF4,0.166667,0.222222,3.100628,2.745494,4.646511,3.410997,1.592850,1.709445,5.655585,7.145301,5.616364,3.692988,3.348563,4.121757
532,LIPF3_LIPF4,0.166667,0.222222,3.100628,2.745494,4.646511,3.410997,1.592850,1.709445,5.655585,7.145301,5.616364,3.692988,3.348563,4.121757
533,LIPF1_LIPF2,0.055556,0.111111,4.022185,6.694994,4.094813,2.623364,2.232299,3.637668,3.884085,2.340909,4.129180,3.364535,2.104218,1.973558
534,RAH1_RAH2,0.285714,0.428571,7.896152,8.692827,1.949869,1.476057,1.850343,1.524082,8.830650,8.764209,4.633951,6.595137,4.016346,6.037426


In [6]:
rand = pd.concat([X_over, y_over], axis=1)
rand

,Channels,SigResp,SigStim,N1RespAvg,N1RespSDV,N2RespAvg,N2RespSDV,P2RespAvg,P2RespSDV,N1StimAvg,N1StimSDV,N2StimAvg,N2StimSDV,P2StimAvg,P2StimSDV,outcome
0,LA9_LA10,0.000000,0.250000,1.530459,1.134169,2.040546,1.369091,1.304096,0.892887,7.178516,10.790559,6.536859,6.383906,5.785279,7.331337,0.0
1,LAH1_LAH2,0.076923,0.250000,7.062089,20.678040,3.320937,2.795194,2.178348,2.563587,7.883750,16.334814,4.921208,3.029861,2.352152,1.803711,1.0
2,LAH8_LAH9,0.307692,0.125000,4.032683,2.995624,3.450575,2.883998,2.376435,2.139730,2.505806,2.164878,2.190308,1.489418,2.033410,2.112725,0.0
3,LPH1_LPH2,0.115385,0.333333,6.522655,17.219698,3.369771,3.506524,2.281413,3.303814,6.505212,8.264811,4.630948,3.574869,4.187831,3.626223,0.0
4,LPH7_LPH8,0.307692,0.083333,7.304854,10.728727,4.610356,4.159990,3.468193,2.645476,2.823084,2.925869,1.606341,1.075715,1.461462,1.012732,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,LIPF3_LIPF4,0.166667,0.222222,3.100628,2.745494,4.646511,3.410997,1.592850,1.709445,5.655585,7.145301,5.616364,3.692988,3.348563,4.121757,1.0
532,LIPF3_LIPF4,0.166667,0.222222,3.100628,2.745494,4.646511,3.410997,1.592850,1.709445,5.655585,7.145301,5.616364,3.692988,3.348563,4.121757,1.0
533,LIPF1_LIPF2,0.055556,0.111111,4.022185,6.694994,4.094813,2.623364,2.232299,3.637668,3.884085,2.340909,4.129180,3.364535,2.104218,1.973558,1.0
534,RAH1_RAH2,0.285714,0.428571,7.896152,8.692827,1.949869,1.476057,1.850343,1.524082,8.830650,8.764209,4.633951,6.595137,4.016346,6.037426,1.0


# SMOTE

In [7]:
counter = Counter(y)
print(counter)
# transform the dataset
oversample = SMOTE()
X_nochannel = newdf.drop(columns = 'Channels')
s_X, s_y = oversample.fit_resample(X_nochannel, y)
# summarize the new class distribution
counter = Counter(s_y)
print(counter)

Counter({0.0: 268, 1.0: 41})
Counter({0.0: 268, 1.0: 268})


In [8]:
s_X

,outcome,SigResp,SigStim,N1RespAvg,N1RespSDV,N2RespAvg,N2RespSDV,P2RespAvg,P2RespSDV,N1StimAvg,N1StimSDV,N2StimAvg,N2StimSDV,P2StimAvg,P2StimSDV
0,0.0,0.000000,0.250000,1.530459,1.134169,2.040546,1.369091,1.304096,0.892887,7.178516,10.790559,6.536859,6.383906,5.785279,7.331337
1,1.0,0.076923,0.250000,7.062089,20.678040,3.320937,2.795194,2.178348,2.563587,7.883750,16.334814,4.921208,3.029861,2.352152,1.803711
2,0.0,0.307692,0.125000,4.032683,2.995624,3.450575,2.883998,2.376435,2.139730,2.505806,2.164878,2.190308,1.489418,2.033410,2.112725
3,0.0,0.115385,0.333333,6.522655,17.219698,3.369771,3.506524,2.281413,3.303814,6.505212,8.264811,4.630948,3.574869,4.187831,3.626223
4,0.0,0.307692,0.083333,7.304854,10.728727,4.610356,4.159990,3.468193,2.645476,2.823084,2.925869,1.606341,1.075715,1.461462,1.012732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,1.0,0.063522,0.051611,3.535630,4.001643,2.262773,1.326654,2.217206,1.914759,3.191200,2.151470,3.021708,1.884339,1.982139,1.518433
532,1.0,0.044279,0.257944,4.009827,8.666252,3.831978,3.392428,2.054736,3.320764,11.075863,23.921535,8.066066,10.973248,5.789808,5.998993
533,1.0,0.000000,0.241767,1.928175,1.183718,3.886967,2.480997,2.448014,2.105060,3.908595,3.216111,4.386417,4.698911,5.403601,3.311957
534,1.0,0.032847,0.084791,3.223252,8.576563,2.313939,2.638729,1.160565,2.369800,6.963412,23.301185,4.563743,9.579903,2.371206,5.977882


In [9]:
smote = pd.concat([s_X, s_y], axis=1)
smote

,outcome,SigResp,SigStim,N1RespAvg,N1RespSDV,N2RespAvg,N2RespSDV,P2RespAvg,P2RespSDV,N1StimAvg,N1StimSDV,N2StimAvg,N2StimSDV,P2StimAvg,P2StimSDV,outcome
0,0.0,0.000000,0.250000,1.530459,1.134169,2.040546,1.369091,1.304096,0.892887,7.178516,10.790559,6.536859,6.383906,5.785279,7.331337,0.0
1,1.0,0.076923,0.250000,7.062089,20.678040,3.320937,2.795194,2.178348,2.563587,7.883750,16.334814,4.921208,3.029861,2.352152,1.803711,1.0
2,0.0,0.307692,0.125000,4.032683,2.995624,3.450575,2.883998,2.376435,2.139730,2.505806,2.164878,2.190308,1.489418,2.033410,2.112725,0.0
3,0.0,0.115385,0.333333,6.522655,17.219698,3.369771,3.506524,2.281413,3.303814,6.505212,8.264811,4.630948,3.574869,4.187831,3.626223,0.0
4,0.0,0.307692,0.083333,7.304854,10.728727,4.610356,4.159990,3.468193,2.645476,2.823084,2.925869,1.606341,1.075715,1.461462,1.012732,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,1.0,0.063522,0.051611,3.535630,4.001643,2.262773,1.326654,2.217206,1.914759,3.191200,2.151470,3.021708,1.884339,1.982139,1.518433,1.0
532,1.0,0.044279,0.257944,4.009827,8.666252,3.831978,3.392428,2.054736,3.320764,11.075863,23.921535,8.066066,10.973248,5.789808,5.998993,1.0
533,1.0,0.000000,0.241767,1.928175,1.183718,3.886967,2.480997,2.448014,2.105060,3.908595,3.216111,4.386417,4.698911,5.403601,3.311957,1.0
534,1.0,0.032847,0.084791,3.223252,8.576563,2.313939,2.638729,1.160565,2.369800,6.963412,23.301185,4.563743,9.579903,2.371206,5.977882,1.0


In [21]:
#network feature example
netdf = pd.read_csv("networkfeatures1.csv").iloc[: , 1:]
ynet = netdf['outcome']
Xnet = netdf.drop(columns = ['outcome', 'Channels'])

In [22]:
counter = Counter(ynet)
print(counter)
# transform the dataset
oversample = SMOTE()
snet_X, snet_y = oversample.fit_resample(Xnet, ynet)
# summarize the new class distribution
counter = Counter(snet_y)
print(counter)

Counter({0.0: 259, 1.0: 41})
Counter({0.0: 259, 1.0: 259})


In [23]:
snet = pd.concat([snet_X, snet_y], axis=1)
snet

,SigResp,SigStim,N1RespAvg,N1RespSDV,N2RespAvg,N2RespSDV,P2RespAvg,P2RespSDV,N1StimAvg,N1StimSDV,N2StimAvg,N2StimSDV,P2StimAvg,P2StimSDV,InDegree,OutDegree,EV,Closeness,outcome
0,0.000000,0.250000,1.530459,1.134169,2.040546,1.369091,1.304096,0.892887,7.178516,10.790559,6.536859,6.383906,5.785279,7.331337,0.000000,0.222222,5.165334e-14,0.000000,0.0
1,0.076923,0.250000,7.062089,20.678040,3.320937,2.795194,2.178348,2.563587,7.883750,16.334814,4.921208,3.029861,2.352152,1.803711,0.074074,0.222222,7.960607e-04,0.177778,1.0
2,0.307692,0.125000,4.032683,2.995624,3.450575,2.883998,2.376435,2.139730,2.505806,2.164878,2.190308,1.489418,2.033410,2.112725,0.296296,0.111111,1.590588e-01,0.484848,0.0
3,0.115385,0.333333,6.522655,17.219698,3.369771,3.506524,2.281413,3.303814,6.505212,8.264811,4.630948,3.574869,4.187831,3.626223,0.111111,0.296296,4.743270e-03,0.222222,0.0
4,0.307692,0.083333,7.304854,10.728727,4.610356,4.159990,3.468193,2.645476,2.823084,2.925869,1.606341,1.075715,1.461462,1.012732,0.296296,0.074074,1.857524e-01,0.484848,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,0.261219,0.168355,6.480518,8.492117,8.561678,14.083507,2.371072,1.881695,13.766706,10.235608,16.404744,7.692749,11.307488,6.319550,0.076914,0.159736,4.206806e-02,0.341627,1.0
514,0.166452,0.064022,4.542326,7.924642,4.612987,6.289535,3.131446,5.825017,2.757989,3.029179,2.675029,1.689523,2.095544,1.665347,0.160115,0.056584,1.410437e-01,0.409781,1.0
515,0.134652,0.179536,2.921745,2.816490,3.345290,2.247192,1.674852,1.203203,5.667716,10.447670,4.422798,4.361353,3.352322,5.896310,0.128689,0.171585,5.014004e-02,0.417122,1.0
516,0.092351,0.218976,7.637520,21.427853,3.799543,4.550446,2.381700,3.231278,9.348827,21.672011,5.072343,4.243922,2.525225,2.747551,0.123387,0.215041,6.357363e-04,0.179789,1.0


# sklearn resample

In [10]:
counter = Counter(y)
print(counter)
num = abs(counter[0] - counter[1])
X_resample, y_resample = resample(X[y == 1], y[y == 1],replace=True,n_samples=num)

#append oversampled minority class to original data
X_balanced = pd.concat([X, X_resample], ignore_index=True)
y_balanced = pd.concat([y, y_resample], ignore_index=True)
print(Counter(y_balanced))

Counter({0.0: 268, 1.0: 41})
Counter({0.0: 268, 1.0: 268})


In [11]:
resample = pd.concat([X_balanced, y_balanced], axis=1)
resample

,Channels,SigResp,SigStim,N1RespAvg,N1RespSDV,N2RespAvg,N2RespSDV,P2RespAvg,P2RespSDV,N1StimAvg,N1StimSDV,N2StimAvg,N2StimSDV,P2StimAvg,P2StimSDV,outcome
0,LA9_LA10,0.000000,0.250000,1.530459,1.134169,2.040546,1.369091,1.304096,0.892887,7.178516,10.790559,6.536859,6.383906,5.785279,7.331337,0.0
1,LAH1_LAH2,0.076923,0.250000,7.062089,20.678040,3.320937,2.795194,2.178348,2.563587,7.883750,16.334814,4.921208,3.029861,2.352152,1.803711,1.0
2,LAH8_LAH9,0.307692,0.125000,4.032683,2.995624,3.450575,2.883998,2.376435,2.139730,2.505806,2.164878,2.190308,1.489418,2.033410,2.112725,0.0
3,LPH1_LPH2,0.115385,0.333333,6.522655,17.219698,3.369771,3.506524,2.281413,3.303814,6.505212,8.264811,4.630948,3.574869,4.187831,3.626223,0.0
4,LPH7_LPH8,0.307692,0.083333,7.304854,10.728727,4.610356,4.159990,3.468193,2.645476,2.823084,2.925869,1.606341,1.075715,1.461462,1.012732,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,LIPF1_LIPF2,0.055556,0.111111,4.022185,6.694994,4.094813,2.623364,2.232299,3.637668,3.884085,2.340909,4.129180,3.364535,2.104218,1.973558,1.0
532,RAH1_RAH2,0.115385,0.153846,2.814085,2.859219,2.562153,1.546758,1.724205,0.898522,5.675017,12.435196,3.704452,4.763608,3.354584,6.964322,1.0
533,RAH1_RAH2,0.285714,0.428571,7.896152,8.692827,1.949869,1.476057,1.850343,1.524082,8.830650,8.764209,4.633951,6.595137,4.016346,6.037426,1.0
534,RMI4_RMI5,0.000000,0.300000,1.811468,1.103896,4.436461,2.844080,2.596925,2.295218,4.084766,3.457806,4.846185,5.418012,6.225441,3.760152,1.0


# Convert to CSV

In [24]:
rand.to_csv('/Users/samanthatam/Documents/year4/fall/PCM/random_oversample.csv')
smote.to_csv('/Users/samanthatam/Documents/year4/fall/PCM/smote.csv')
resample.to_csv('/Users/samanthatam/Documents/year4/fall/PCM/resample.csv')
snet.to_csv('/Users/samanthatam/Documents/year4/fall/PCM/network.csv')